In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:


import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
import os
import numpy as np

from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
os.listdir('../input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled/Zhu_Rongji')

img_fld = ImageFolder('../input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled', transform=ToTensor())

In [ ]:
#so we have a discriminator and we have a generator
#first we will create a discriminator - i will make it feedforward for now

img_size = (250 ** 2) * 3
D = nn.Sequential(
    nn.Linear(img_size, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 128),
    nn.LeakyReLU(0.2),
    nn.Linear(128, 1),
    nn.Sigmoid()
)

#this is the generator model. It takes in a random input vector and then it 

G = nn.Sequential(
    nn.Linear(64, 128),
    nn.ReLU(),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Linear(128, img_size),
    nn.Tanh()
    
)


random_input_vector = torch.randn(2,64)
def mgraph(random_input_vector):
    tst_out = (G(random_input_vector.to(torch.device("cuda"))))
    gg = tst_out.reshape(-1, 250, 250)
    gg = gg.reshape(2,3,250, 250)
    gg = gg[0].permute(1,2,0).cpu().detach()
    import matplotlib.pyplot as plt
    plt.imshow(gg)
# mgraph(random_input_vector)

In [ ]:
#we will have a batch size of 33, because it is divisible by 13233
#there is no validation set, becuase we are just generating things not actually like 
loss_fn = nn.BCELoss()
d_opt = torch.optim.Adam(D.parameters(), lr = 2e-4)
g_opt = torch.optim.Adam(G.parameters(), lr = 2e-4)
def reset_grad():
    d_opt.zero_grad()
    g_opt.zero_grad()
    
train_dl = DataLoader(img_fld, 33, shuffle=True)

real_labels = torch.ones(33,1).to(torch.device('cuda'))
fake_labels = torch.zeros(33,1).to(torch.device('cuda'))
print(len(train_dl))
loss_fn = nn.BCELoss()
D = D.to(torch.device('cuda'))
G = G.to(torch.device('cuda'))
def train_dis(xb):  
    x = xb.view(xb.size(0), -1).to(torch.device('cuda'))

    out = D(x)
    d_loss_real = loss_fn(out, real_labels)
    riv = torch.randn(33,64).to(torch.device('cuda'))
    fake_images = G(riv)
    outputs = D(fake_images)


    d_loss_fake = loss_fn(outputs, fake_labels)
    d_loss = d_loss_real + d_loss_fake
    
    reset_grad() #we need to reset the gradient before we calculate the loss so it doesnt append to the total gradient
    d_loss.backward()
    d_opt.step()
    
    

def train_gen():
    z = torch.randn(33,64).to(torch.device('cuda'))
    fake_images = G(z)
    lbl = torch.ones(33,1).to(torch.device('cuda'))
#     print(D(fake_images))
    g_loss = loss_fn(D(fake_images), lbl)
#     print(g_loss)
    reset_grad()
    g_loss.backward()
    g_opt.step()


In [ ]:
d_opt = torch.optim.Adam(D.parameters(), lr = 2e-4)
g_opt = torch.optim.Adam(G.parameters(), lr = 2e-4)

In [ ]:
loss_fn = nn.BCELoss()

In [ ]:
# #the discriminator kinda worked

#training the discriminator is hella ezy
# for images,_ in train_dl:
    
#     img = images.view(images.size(0), -1)
#     train_dis(images.to(torch.device("cuda")))


In [ ]:
#you know actually, training on the gpu is soooo much faster, like a big difference rather than training on the cpu.
for xb, _ in train_dl:
    x = xb.view(xb.size(0), -1).to(torch.device("cuda"))
    print(xb.size(0))
    print(D(x))
    break

In [ ]:
num_epochs = 2

for epoch in range(num_epochs):
    for images,_ in train_dl:
        train_dis(images.to(torch.device('cuda')))
        train_gen()
        
    

In [ ]:
for xb, _ in train_dl:
    xb = xb.view(xb.size(0), -1)
    print(D(xb.to(torch.device('cuda'))))
    break

In [ ]:
rand_inp_vector = torch.randn(1,64).to(torch.device('cuda'))
print(rand_inp_vector)

sh = (G(rand_inp_vector.to(torch.device("cuda"))))

sh = sh.cpu().detach()

sh = sh.reshape(-1, 250, 250)

sh = sh.permute(1,2,0)

plt.imshow(sh)


print(sh.shape)
# gg = tst_out.reshape(-1, 250, 250)
# gg = gg.reshape(2,3,250, 250)
# gg = gg[0].permute(1,2,0).cpu().detach()
# import matplotlib.pyplot as plt
# plt.imshow(gg)

img = (img_fld[0][0]).permute(1,2,0)

# plt.imshow(img)

